In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn import preprocessing, model_selection, metrics
from sklearn import linear_model, naive_bayes, ensemble
# from keras.models import Sequential
# from keras.layers import Dense, Activation 

%matplotlib inline

plt.rcParams["figure.figsize"] = (10,8)
plt.rcParams["axes.titlesize"] = 15
pd.set_option("max_colwidth", 0)

In [2]:
! ls "../../../datasets/lending-club/"

LCDataDictionary.xlsx loans                 rejects


In [3]:
data_dictionary = pd.read_excel("../../../datasets/lending-club/LCDataDictionary.xlsx")
data_dictionary

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


# About the data

- Source: [Lending Club](https://www.lendingclub.com/info/download-data.action)
- Loan stats: details of loans given out, including current status of loan
- Reject stats: details of loan applications rejected

# Problem statement

2 possible problem statements:
- Can we predict whether a loan application will be accepted or rejected?
- Can we predict whether a lender will default on his loan?

# Data management

- [Separate notebook]()
- Different time periods have been saved as different files at source
- Combine all our data into a single data source
- The 2 problem statements will use different portions of the data
    - Additional cleaning will be done in the respective notebooks

# Loan application approval

[Prediction approvals on loan applications]()

## Data cleaning

In [149]:
app_df.dropna(how="all", inplace=True)

In [150]:
app_df = app_df[app_df.loan_amnt > 0]

In [151]:
app_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8013610 entries, 0 to 1096203
Data columns (total 7 columns):
loan_amnt      float64
purpose        object
dti            object
zip_code       object
addr_state     object
emp_length     object
policy_code    float64
dtypes: float64(2), object(5)
memory usage: 489.1+ MB


In [152]:
def remove_percent(x):
    try:
        return float(x.strip("%"))
    except:
        return float(x)

def remove_x(x):
    try:
        return float(x.strip("x"))
    except:
        return float(x)

In [154]:
app_df.loc[:,"dti"] = app_df.dti.apply(remove_percent)

In [155]:
app_df.loc[:,"zip_code"] = app_df.zip_code.apply(remove_x)

In [156]:
app_df.addr_state.unique()

array(['AZ', 'GA', 'IL', 'CA', 'OR', 'NC', 'TX', 'VA', 'MO', 'CT', 'UT',
       'FL', 'NY', 'PA', 'MN', 'NJ', 'KY', 'OH', 'SC', 'RI', 'LA', 'MA',
       'WA', 'WI', 'AL', 'CO', 'KS', 'NV', 'AK', 'MD', 'WV', 'VT', 'MI',
       'DC', 'SD', 'NH', 'AR', 'NM', 'MT', 'HI', 'WY', 'OK', 'DE', 'MS',
       'TN', 'IA', 'NE', 'ID', 'IN', 'ME', 'ND'], dtype=object)

In [157]:
app_df["state"] = app_df["addr_state"]
del app_df["addr_state"]

In [158]:
app_df.emp_length.unique()

# GROUP N/A WITH < 1
app_df.loc[:, "emp_length"] = app_df.emp_length.replace(to_replace=["n/a"], value="< 1 year")

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a'], dtype=object)

In [160]:
app_df.purpose.unique()

array(['credit_card', 'car', 'small_business', 'other', 'wedding',
       'debt_consolidation', 'home_improvement', 'major_purchase',
       'medical', 'moving', 'vacation', 'house', 'renewable_energy',
       'educational', 'Debt consolidation', 'Other', 'Green loan',
       'Credit card refinancing', 'Business Loan', 'Home improvement',
       'Business', 'Major purchase', 'Medical expenses', 'Vacation',
       'Moving and relocation', 'Home buying', 'Car financing'], dtype=object)

In [161]:
def sub_purpose(x):
    if "credit" in x.lower():
        return "credit_card"
    elif "moving" in x.lower():
        return "moving"
    elif "car" in x.lower():
        return "car"
    elif "debt" in x.lower():
        return "debt_consolidation"
    elif "improv" in x.lower():
        return "home_improvement"
    elif "home" in x.lower() or "house" in x.lower():
        return "house_purchase"
    elif " vacation" in x.lower():
        return "vacation"
    elif "medical" in x.lower():
        return "medical"
    elif "business" in x.lower():
        return "business"
    elif "major" in x.lower():
        return "major_purchase"
    elif "energy" in x.lower() or "green" in x.lower():
        return "renewable_energy"
    else:
        return "other"

In [162]:
app_df["purpose"] = app_df.purpose.apply(sub_purpose)

In [163]:
app_df = pd.get_dummies(app_df, columns=["state", "emp_length", "purpose"])
app_df.columns = [i.replace(" ", "_") for i in app_df.columns]

In [164]:
app_df.head()

,loan_amnt,dti,zip_code,policy_code,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,...,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house_purchase,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy
0,5000.0,27.65,860.0,1.0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2500.0,1.00,309.0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2400.0,8.72,606.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000.0,20.00,917.0,1.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,3000.0,17.94,972.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [167]:
with open("app_df_clean.pkl", "wb") as f:
    pickle.dump(app_df, f)

## Modeling

In [2]:
with open("app_df_clean.pkl", "rb") as f:
    app_df = pickle.load(f)

In [29]:
app_df = app_df.dropna().reset_index(drop=True)

In [30]:
X = app_df[[i for i in app_df.columns if i!= "policy_code"]]
y = app_df["policy_code"]

In [31]:
y.value_counts()

0.0    7596274
1.0    340280 
2.0    76734  
Name: policy_code, dtype: int64

In [32]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y, train_size=0.9)

In [33]:
X_train.reset_index(inplace=True, drop=True)

In [34]:
nb = naive_bayes.GaussianNB()

In [61]:
nb.partial_fit(X_train.iloc[:1000000], y_train[:1000000], classes=y.unique())

GaussianNB(priors=None)

In [62]:
nb.score(X_test, y_test)

0.31117181582096742

In [78]:
def train_nb(start, stop):
    nb.partial_fit(X_train.iloc[start:stop], y_train[start:stop])

In [82]:
for i in range(20):
    train_nb(None,1000000)
    train_nb(1000000,2000000)
    train_nb(2000000,3000000)
    train_nb(3000000,4000000)
    train_nb(4000000,5000000)
    train_nb(5000000,6000000)
    train_nb(6000000,None)
nb.score(X_test,y_test)

0.34461500831743269

0.34427058049814746

In [7]:
X_train.shape

(801361, 76)